In [1]:
import requests

In [2]:
from bs4 import BeautifulSoup

In [3]:
import pandas as pd

In [4]:
years = list(range(2022, 2018, -1))

In [5]:
all_matches = []

In [28]:
groups_url = 'https://fbref.com/en/comps/1/1711/2022-FIFA-World-Cup-Stats' 

In [32]:
import time

data = requests.get(groups_url)
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0:9]
links = [st.find_all('a') for st in standings_table]

# get all the links of the groups in a single list 
all_links = [link for groups in links for link in groups]
all_links = [l.get('href') for l in all_links]
all_list = [l for l in all_links if '/squads/' in l]
team_urls = [f'https://fbref.com{l}' for l in all_list]
    
for team_url in team_urls:
    team_name = team_url.split('/')[-1].replace('-Men-Stats', '')

    for year in years:
        if year == 2022:
            season = 2022
        else:
            season = team_url.split('/')[-2]
        print(team_name ,year, data)
        data = requests.get(team_url)
        #create matches df
        matches = pd.read_html(data.text, match='Scores & Fixtures')[0]

        #get and create shooting df
        soup = BeautifulSoup(data.text)
        links = soup.find_all('a')
        links = [l.get('href') for l in links]
        links = [l for l in links if l and '/all_comps/shooting/' in l]
        data = requests.get(f'https://fbref.com{links[0]}')
        shooting = pd.read_html(data.text, match='Shooting')[0]
        shooting.columns = shooting.columns.droplevel()

        # somtimes shooting values dont exist for some teams
        try:
            team_data = matches.merge(shooting[['Date', 'Sh', 'SoT', 'PK', 'PKatt']], on ='Date')
        except ValueError:
            continue
            
        previous_season = soup.select('div.prevnext')[0]
        previous_season = previous_season.select('a')[0].get('href')
        team_url = f'https://fbref.com{previous_season}'
        team_data['Season'] = season
        team_data['Team'] = team_name
        all_matches.append(team_data)
        time.sleep(10)

Ecuador 2022 <Response [200]>
Netherlands 2022 <Response [200]>
Senegal 2022 <Response [200]>
Qatar 2022 <Response [200]>
England 2022 <Response [200]>
IR-Iran 2022 <Response [200]>
United-States 2022 <Response [200]>
Wales 2022 <Response [200]>
Argentina 2022 <Response [200]>
Mexico 2022 <Response [200]>
Poland 2022 <Response [200]>
Saudi-Arabia 2022 <Response [200]>
Australia 2022 <Response [200]>
Denmark 2022 <Response [200]>
France 2022 <Response [200]>
Tunisia 2022 <Response [200]>
Costa-Rica 2022 <Response [200]>
Germany 2022 <Response [200]>
Japan 2022 <Response [200]>
Spain 2022 <Response [200]>
Belgium 2022 <Response [200]>
Canada 2022 <Response [200]>
Croatia 2022 <Response [200]>
Morocco 2022 <Response [200]>
Brazil 2022 <Response [200]>
Cameroon 2022 <Response [200]>
Serbia 2022 <Response [200]>
Switzerland 2022 <Response [200]>
Ghana 2022 <Response [200]>
Korea-Republic 2022 <Response [200]>
Korea-Republic 2021 <Response [200]>
Korea-Republic 2020 <Response [200]>
Korea-Re

In [33]:
match_df = pd.concat(all_matches)

In [34]:
match_df.columns = [c.lower() for c in match_df.columns]

In [35]:
len(match_df[match_df['season']==2022].team.unique())

32

In [60]:
match_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1959 entries, 0 to 28
Data columns (total 25 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          1959 non-null   object 
 1   time          1959 non-null   object 
 2   comp          1959 non-null   object 
 3   round         1959 non-null   object 
 4   day           1959 non-null   object 
 5   venue         1959 non-null   object 
 6   result        1958 non-null   object 
 7   gf            1958 non-null   object 
 8   ga            1958 non-null   object 
 9   opponent      1959 non-null   object 
 10  poss          1049 non-null   float64
 11  attendance    939 non-null    float64
 12  captain       1852 non-null   object 
 13  formation     1843 non-null   object 
 14  referee       1955 non-null   object 
 15  match report  1959 non-null   object 
 16  notes         64 non-null     object 
 17  sh            1050 non-null   float64
 18  sot           1050 non-null   

In [75]:
match_df.to_csv('matches.csv')